第三題:<br>
爬取一筆公司營業地址，在財政部稅務入口網，選擇依據營業人統一編號查詢頁面會有圖片驗證碼，使用AI 學習方式或其他方式可以進入爬找資料。

財政部稅務入口: https://www.etax.nat.gov.tw/cbes/web/CBES113W1


In [ ]:
# !kill -9 -1
!sudo apt update
!sudo apt install chromium-chromedriver
!pip3 install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [96.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 P

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By as BY
from selenium.webdriver.support import ui as UI
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

In [ ]:
def initSelenium(proxy):
  '''
  init selenium and setting options
  '''
  #selenium options setting
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("--disable-infobars")
  # add proxy
  if proxy != None:
    chrome_options.add_argument('--proxy-server=http://{}'.format(proxy))

  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  #set browser loading timeout
  driver.set_page_load_timeout(120)

  return driver

In [ ]:
def getMailAddr(driver):
  '''
  get free 10 Minute Mail address
  '''
  driver.get('https://10minutemail.net/')
  UI.WebDriverWait(driver,15).until(lambda driver: driver.find_element_by_id("fe_text"))
  # use 10minutes email get captcha
  ten_minute_mail = driver.find_element_by_id("fe_text").get_attribute('value')
  
  return driver,ten_minute_mail


In [ ]:
def sendCaptcha2Mail(driver,ten_minute_mail):
  '''
  use 10 Minute Mail to receive captcha
  '''
  driver.execute_script("window.open('https://www.etax.nat.gov.tw/cbes/web/CBES113W1_1')")
  driver.switch_to.window(driver.window_handles[1])
  UI.WebDriverWait(driver,10).until(lambda driver: driver.find_element_by_xpath("//input[@id='vatId']"))
  driver.find_element_by_xpath("//a[@title='驗證碼']").click()
  time.sleep(1)

  driver.find_element_by_xpath("//input[@id='captchaReceiverMail']").send_keys(ten_minute_mail)
  driver.find_element_by_xpath("//button[@id='btn_submit']").click()
  return driver

In [ ]:
def getCaptchaCode(driver):
  '''
  get captcha
  '''
  driver.switch_to.window(driver.window_handles[0])
  UI.WebDriverWait(driver,120).until(EC.text_to_be_present_in_element(('id','maillist'),'財政資訊中心'))
  driver.find_element_by_xpath("//td[text()='財政資訊中心 <webmail@etax.nat.gov.tw>']").click()
  code = driver.find_element_by_class_name('mailinhtml').text.split('：',1)[1]

  return driver,code

In [ ]:
def getCompanyAddr(driver,gui,code):
  '''
  get company data
  '''
  driver.switch_to.window(driver.window_handles[1])
  driver.find_element_by_xpath("//input[@id='vatId']").send_keys(gui)
  driver.find_element_by_id('captcha').send_keys(code)
  driver.find_element_by_xpath("//input[@type='submit']").click()
  UI.WebDriverWait(driver,15).until(lambda driver: driver.find_element_by_id("address"))
  company = driver.find_element_by_id('businessName').text
  addr = driver.find_element_by_id('address').text
  
  return driver,(company,addr)

In [ ]:
if __name__ == '__main__':
  # gui: 要查詢的公司統編
  gui = '05155853'
  driver = initSelenium(proxy=None)
  driver,ten_minute_mail = getMailAddr(driver)
  driver = sendCaptcha2Mail(driver,ten_minute_mail)
  # 因為要等待收信，所以會比較久
  driver,code = getCaptchaCode(driver)
  driver,result = getCompanyAddr(driver,gui,code)
  print(result)
  driver.quit()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


('研華股份有限公司', '臺北市內湖區湖元里瑞光路２６巷２０弄１號１樓')
